In [9]:
!pip install bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [12]:
# Get the data
df = pd.read_csv('data/BrasilSerieA.csv')
df.head()

,Rk,Player,Gls/90,G/Sh,G/SoT,SoT%,SoT/90,Sh/90,G-PK/90,PK/90,...,Team,Comp,MP,Min,90s,Starts,Subs,unSub,Pos,-9999
0,1,Gabriel Barbosa,0.9,0.18,0.35,50.9,2.0,4.0,0.7,0.2,...,Flamengo,br Série A,29,2537,28.2,29,0,1,FW,72d0e1b6
1,2,Pedro,0.8,0.21,0.46,46.2,1.8,4.0,0.8,0.0,...,Flamengo,br Série A,24,1174,13.0,12,12,4,FW,1ab6fad3
2,3,Thiago Galhardo,0.8,0.17,0.39,43.8,1.4,3.1,0.5,0.3,...,Internacional,br Série A,29,1859,20.7,20,9,0,FW,766f48dd
3,4,Germán Cano,0.8,0.20,0.41,48.5,1.8,3.8,0.8,0.0,...,Fluminense,br Série A,38,3081,34.2,37,1,0,FW,9ccf4e14
4,5,Bruno Henrique,0.7,0.22,0.47,46.4,1.6,3.4,0.7,0.0,...,Flamengo,br Série A,33,2570,28.6,29,4,0,FWMF,5edff24f


In [18]:
#Drop last column
#df = df.drop(df.columns[-1], axis=1)
pd.set_option('display.max_columns', None)
df.head()

,Rk,Player,Gls/90,G/Sh,G/SoT,SoT%,SoT/90,Sh/90,G-PK/90,PK/90,PKatt/90,Dist,Season,Age,Nation,Team,Comp,MP,Min,90s,Starts,Subs,unSub,Pos
0,1,Gabriel Barbosa,0.9,0.18,0.35,50.9,2.0,4.0,0.7,0.2,0.2,16.1,2019,22,br BRA,Flamengo,br Série A,29,2537,28.2,29,0,1,FW
1,2,Pedro,0.8,0.21,0.46,46.2,1.8,4.0,0.8,0.0,0.1,10.5,2022,24,br BRA,Flamengo,br Série A,24,1174,13.0,12,12,4,FW
2,3,Thiago Galhardo,0.8,0.17,0.39,43.8,1.4,3.1,0.5,0.3,0.3,15.5,2020,30,br BRA,Internacional,br Série A,29,1859,20.7,20,9,0,FW
3,4,Germán Cano,0.8,0.20,0.41,48.5,1.8,3.8,0.8,0.0,0.0,16.6,2022,33,ar ARG,Fluminense,br Série A,38,3081,34.2,37,1,0,FW
4,5,Bruno Henrique,0.7,0.22,0.47,46.4,1.6,3.4,0.7,0.0,0.0,14.6,2019,28,br BRA,Flamengo,br Série A,33,2570,28.6,29,4,0,FWMF


In [20]:
# Read player column and season column, if a player has more than one entry, calculate the average of each value and enter it into the dataframe. Remove the duplicates.
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
fixed_df = df.groupby(['Player', 'Season'], as_index=False)[numeric_cols].mean()
# Calculate the average of each value
fixed_df = fixed_df.groupby('Player', as_index=False)[numeric_cols].mean()
# Remove the duplicates
fixed_df = fixed_df.drop_duplicates()
fixed_df.head()

,Player,Rk,Gls/90,G/Sh,G/SoT,SoT%,SoT/90,Sh/90,G-PK/90,PK/90,PKatt/90,Dist,Season,Age,MP,Min,90s,Starts,Subs,unSub
0,Abel Hernández,208.0,0.300,0.1300,0.3300,37.50,0.900,2.3,0.300,0.00,0.00,12.10,2020.0,29.0,23.0,925.0,10.30,10.00,13.00,5.00
1,Adailson Dadá,585.0,0.100,0.0400,0.1800,22.90,0.400,1.7,0.100,0.00,0.00,23.10,2022.0,24.0,33.0,2508.0,27.90,30.00,3.00,0.00
2,Ademir Santos,259.5,0.275,0.0975,0.2675,37.15,0.875,2.4,0.225,0.05,0.05,17.95,2022.5,26.5,32.5,1732.5,19.25,18.25,14.25,1.25
3,Adrián Martínez,147.0,0.400,0.1500,0.4000,37.00,0.900,2.4,0.400,0.00,0.00,13.10,2022.0,29.0,22.0,1000.0,11.10,10.00,12.00,4.00
4,Adson,382.0,0.150,0.0950,0.2900,34.80,0.650,1.9,0.150,0.00,0.00,19.25,2023.0,22.0,22.5,1231.5,13.70,13.50,9.00,2.00


In [21]:
#Drop Rk, Season columns
fixed_df = fixed_df.drop(['Rk', 'Season'], axis=1)
fixed_df.head()

,Player,Gls/90,G/Sh,G/SoT,SoT%,SoT/90,Sh/90,G-PK/90,PK/90,PKatt/90,Dist,Age,MP,Min,90s,Starts,Subs,unSub
0,Abel Hernández,0.300,0.1300,0.3300,37.50,0.900,2.3,0.300,0.00,0.00,12.10,29.0,23.0,925.0,10.30,10.00,13.00,5.00
1,Adailson Dadá,0.100,0.0400,0.1800,22.90,0.400,1.7,0.100,0.00,0.00,23.10,24.0,33.0,2508.0,27.90,30.00,3.00,0.00
2,Ademir Santos,0.275,0.0975,0.2675,37.15,0.875,2.4,0.225,0.05,0.05,17.95,26.5,32.5,1732.5,19.25,18.25,14.25,1.25
3,Adrián Martínez,0.400,0.1500,0.4000,37.00,0.900,2.4,0.400,0.00,0.00,13.10,29.0,22.0,1000.0,11.10,10.00,12.00,4.00
4,Adson,0.150,0.0950,0.2900,34.80,0.650,1.9,0.150,0.00,0.00,19.25,22.0,22.5,1231.5,13.70,13.50,9.00,2.00


In [22]:
#Save dataframe as csv
fixed_df.to_csv('data/Fixed_Brasil_SerieA.csv', index=False)
fixed_df.head()

,Player,Gls/90,G/Sh,G/SoT,SoT%,SoT/90,Sh/90,G-PK/90,PK/90,PKatt/90,Dist,Age,MP,Min,90s,Starts,Subs,unSub
0,Abel Hernández,0.300,0.1300,0.3300,37.50,0.900,2.3,0.300,0.00,0.00,12.10,29.0,23.0,925.0,10.30,10.00,13.00,5.00
1,Adailson Dadá,0.100,0.0400,0.1800,22.90,0.400,1.7,0.100,0.00,0.00,23.10,24.0,33.0,2508.0,27.90,30.00,3.00,0.00
2,Ademir Santos,0.275,0.0975,0.2675,37.15,0.875,2.4,0.225,0.05,0.05,17.95,26.5,32.5,1732.5,19.25,18.25,14.25,1.25
3,Adrián Martínez,0.400,0.1500,0.4000,37.00,0.900,2.4,0.400,0.00,0.00,13.10,29.0,22.0,1000.0,11.10,10.00,12.00,4.00
4,Adson,0.150,0.0950,0.2900,34.80,0.650,1.9,0.150,0.00,0.00,19.25,22.0,22.5,1231.5,13.70,13.50,9.00,2.00
